In [ ]:
import sys
import os.path
import pikepdf
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

Path("temp").mkdir(parents=True, exist_ok=True)
Path("data").mkdir(parents=True, exist_ok=True)

In [ ]:
if (os.path.exists("data/meta.csv")):
    os.remove("data/meta.csv")

In [ ]:
pdfurl = pd.read_csv("data/downloadedpdf.csv").fillna(value = "")
pdfurl

In [ ]:
#loop
for index, row in pdfurl.iterrows():
    
    unique = row['URL'].removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
    unique = unique.removesuffix("\n")
    metadict = {"Identifier":unique}   
    pfad = "temp/" + unique + ".pdf"
    if((row['Error'] == "")):
        #Falls beim Download kein Error aufgetreten ist: Probier die Datei zu öffnen
        try:
            pdf = pikepdf.Pdf.open(pfad)           
        except pikepdf._qpdf.PasswordError:
            print("This pdf requires a password.")
            metadict.update({"Error":'PasswordProtected'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                #oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)
                oldframe = newframe
            else:
                frame.to_csv("data/meta.csv", index=False)
                oldframe = frame
            metadict.clear           
            continue
        except:
            print("An Error occured while opening.")
            metadict.update({"Error":'OpeningError'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                #oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)
                oldframe = newframe
            else:
                frame.to_csv("data/meta.csv", index=False)
                oldframe = frame
            metadict.clear
            continue
        
        #Read Metadata from pdfs docinfo
        docinfo = pdf.docinfo

        try:
            docinfo["/Author"]
        except KeyError:
            metadict.update({"Author":""})
        else:
            metadict.update({"Author":str(docinfo["/Author"]).replace("\n", "")\
                                                             .replace("\r", "")\
                                                             .replace("\00", "")})

        try:
            docinfo["/Keywords"]
        except KeyError:
            metadict.update({"Keywords":""})
        else:
            metadict.update({"Keywords":str(docinfo["/Keywords"]).replace("\n", "")\
                                                                 .replace("\r", "")\
                                                                 .replace("\00", "")\
                                                                 .replace(";", ",")\
                                                                 .replace(", ", ",")\
                                                                 .replace(" ", "_")})
            
            
        try:
            docinfo["/CreationDate"]
        except KeyError:
            metadict.update({"Datum der Erstellung":""})
        else:
            creation = str(docinfo["/CreationDate"]).replace("'", "")
            
            if(len(creation) > 12):
                if(creation[0] != 'D'):
                    creation = 'D:' + creation
                if(len(creation) >=17):
                    if(creation[16]== 'Z'):
                        creation = creation.removesuffix("0000")
                elif(len(creation) == 16):
                    creation = creation + 'Z'
                try:
                    date = datetime.strptime(creation, "D:%Y%m%d%H%M%S%z")
                except ValueError:
                    metadict.update({"Datum der Erstellung":""})
                else:
                    metadict.update({"Datum der Erstellung":date.strftime("%Y/%m/%d")}) 
            else:
                metadict.update({"Datum der Erstellung":""})
    
        try:
            docinfo["/Title"]
        except KeyError:
            metadict.update({"Titel":""})
        else:
            metadict.update({"Titel":str(docinfo["/Title"]).replace("\n", "")\
                                                           .replace("\r", "")\
                                                           .replace("\00", "")})

            
        metadict.update({"Seitenzahl":len(pdf.pages)})

        
        if (pdf.pages[0].trimbox[2] - pdf.pages[0].trimbox[0]) < (pdf.pages[0].trimbox[3] - pdf.pages[0].trimbox[1]):
            metadict.update({"Format":"Hochformat"})
        else:
            metadict.update({"Format":"Querformat"})

        metadict.update({"Dateigröße":os.stat(pfad).st_size})
        pdf.close()
    else:
        metadict.update({'Error':row['Error']})

        
    frame = pd.DataFrame(metadict, [0])
    if (os.path.exists("data/meta.csv")):   
        newframe = pd.concat([oldframe, frame])
        newframe.to_csv("data/meta.csv", index=False)
        oldframe = newframe
            
    else:
        frame.to_csv("data/meta.csv", index=False)
        oldframe = frame
        
    metadict.clear
    print("finshed:", index+1)